<a href="https://colab.research.google.com/github/cltl/GRMCAnnotation/blob/main/friends/friends2gmrc-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone the repo, download the necessary files, etc.

In [ ]:
%cd /content
!git clone https://github.com/leolani/cltl-face-all
!cd cltl-face-all/ && pip install .
!pip install omegaconf==2.0.5
!pip install tqdm av

# Download the annotations and stuff
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datatsets-jsons/datasets.json

!gdown --id 1-2LeHC_5Cm2gWWT6vBrVhp8jorbjkN1_
!unzip visual-features.zip
!rm visual-features.zip

!gdown --id 16ck7plW9v9eSHGCs5wuB2AhhufPRt3Wi
!unzip smaller-dataset.zip
!rm smaller-dataset.zip

!ls smaller-dataset/  |  wc -l
!ls visual-features | wc -l

!git clone https://github.com/cltl/GRMCAnnotation.git
!cd GRMCAnnotation && pip install .

## read pre-computed visual features and annotations

In [2]:
import json

with open('datasets.json', 'r') as stream:
    datasets = json.load(stream)

datasets = datasets['large']

from glob import glob
import numpy as np
import os
visual_features = glob('visual-features/*.npy')
visual_features = {os.path.basename(vf).split('.npy')[0] : np.load(vf, allow_pickle=True).item() for vf in visual_features}

## Select a random dialogue. Keep running the cell until you get what you want. Preferably, you want to have a video that includes Joey, Chandler, Monica, Ross, Rachel, or Phoebe.

In [3]:
from glob import glob
import random
import os
import json
import numpy as np
import av
import matplotlib.pyplot as plt
import cv2
import random
from IPython.display import HTML
from base64 import b64encode

VISUAL_FEATURES_PATH = 'visual-features/'
VIDEOS_PATH = "smaller-dataset/"

In [4]:
def sort_nicely(arg):
    max_length = max([len(arg_) for arg_ in arg])
    min_length = min([len(arg_) for arg_ in arg])

    if max_length == min_length:
        return sorted(arg)
    else:
        return sorted([arg_ for arg_ in arg if len(arg_) == min_length]) + sorted([arg_ for arg_ in arg if len(arg_) == max_length])

diautt = random.choice([foo for foo, bar in datasets['train'].items()])
dia = diautt.split('_')[0]
diautts = sorted([diautt for diautt in list(datasets['train'].keys()) if dia in diautt])
diautts = sort_nicely(diautts)

annots = {diautt: datasets['train'][diautt] for diautt in diautts}
vids_path = {diautt: os.path.join(VIDEOS_PATH, diautt) +  '.mp4' for diautt in diautts}
features_path = {diautt: os.path.join(VISUAL_FEATURES_PATH, diautt) +  '.npy' for diautt in diautts}
features = {diautt: np.load(path, allow_pickle=True).item() for diautt, path in features_path.items()}

an = annots[diautts[0]]
vp = vids_path[diautts[0]]
print(an)
mp4 = open(vp,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=1280 controls>
    <source src="%s" type="video/mp4">
</video>
""" % data_url)

{'SrNo': '7737', 'Utterance': 'Wow, you have really pretty feet.', 'Speaker': 'Rick', 'Emotion': 'surprise', 'Sentiment': 'positive', 'Dialogue_ID': '778', 'Utterance_ID': '0', 'Season': '4', 'Episode': '4', 'StartTime': '00:09:11,008', 'EndTime': '00:09:12,968'}


In [5]:
import pickle
from datetime import datetime, timedelta

with open('GRMCAnnotation/friends/friends-time/friends-time.pkl', 'rb') as stream:
    friends_time = pickle.load(stream)

In [ ]:
import os
import av
import numpy as np
import cv2
from tqdm.notebook import tqdm
import time
import csv

from glob import glob
import numpy as np
from cltl_face_all.arcface import calc_angle_distance


THRESHOLDS = {'face': 0.9, 'angle': 1.15}

predefined_faces_= glob(os.path.join('./cltl-face-all/your-faces/*/*.npy'))
predefined_faces = {}

for path in predefined_faces_:
    name = path.split('/')[-2]
    predefined_faces[name] = np.load(path)

os.makedirs(os.path.join(dia, 'image'), exist_ok=True)
os.makedirs(os.path.join(dia, 'text'), exist_ok=True)
os.makedirs(os.path.join(dia, 'audio'), exist_ok=True)

image_grmc = []
chat = []
for diautt in tqdm(diautts):
    annot = annots[diautt]
    vis = visual_features[diautt]
    vidpath = vids_path[diautt]

    season = annot['Season']
    episode = annot['Episode']
    emotion = annot['Emotion']
    sentiment = annot['Sentiment']
    starttime = annot['StartTime']
    endttime = annot['EndTime']
    utterance = annot['Utterance']
    speaker = annot['Speaker']

    hours, minutes, seconds = starttime.split(':')
    seconds, milliseconds = seconds.split(',')

    hours, minutes, seconds, milliseconds = int(hours), int(minutes), int(seconds), int(milliseconds)

    time_datetime = friends_time[season][episode] + timedelta(hours=hours, minutes=minutes, seconds=seconds)
    time_unix = time.mktime(time_datetime.timetuple())
    time_unix_ms = int(time_unix*1000 + milliseconds)

    chat.append([speaker, utterance, time_unix_ms])

    aud = diautt + '.wav'
    !ffmpeg -i $vidpath -q:a 0 -map a $dia/audio/$aud

    container = av.open(vidpath)
    fps = float(container.streams.video[0].average_rate)
    for frame in container.decode(video=0):
        idx = frame.index
        numpy_RGB = np.array(frame.to_image())
        numpy_BGR = cv2.cvtColor(numpy_RGB, cv2.COLOR_RGB2BGR)
        impath = os.path.join(dia, 'image', diautt + f'_frame{str(idx).zfill(5)}.jpg')
        cv2.imwrite(impath, numpy_BGR)

        features = vis[idx]
        # Assume that there is only one unique face per frame.
        for feat in features:
            to_append = {}

            age = feat['age']
            gender = feat['gender']
            bbox = feat['bbox']
            embedding = feat['embedding']
            landmark = feat['landmark']

            if bbox[-1] < THRESHOLDS['face']:
                continue

            if speaker not in list(predefined_faces.keys()):
                continue

            embedding.reshape(1, 512)

            dists = {key: calc_angle_distance(embedding, val) for key, val \
                        in predefined_faces.items()}

            if dists[speaker] < THRESHOLDS['angle']:
                to_append['files'] = impath
                to_append['bbox'] = bbox.astype(int).tolist()[:4]
                to_append['speaker'] = speaker
                image_grmc.append(to_append)

with open(os.path.join(dia, 'text', 'chat.csv'), 'w') as stream:
    stream.write('speaker,utterance,time\n')

    for line in chat:
        speaker, utterance, time_unix_ms = line
        stream.write(speaker)
        stream.write(',')
        stream.write(f"\"{utterance}\"")
        stream.write(',')
        stream.write(str(time_unix_ms))
        stream.write('\n')    

In [37]:
with open(os.path.join(dia, 'image.json'), 'w') as stream:
    json.dump(image_grmc, stream)

!touch $dia/$dia.json